In [1]:
%%configure -f
{ "name":"Spark-to-Cosmos_DB_Connector",  
  "conf": {
    "spark.jars.packages": "com.microsoft.azure:azure-cosmosdb-spark_2.4.0_2.11:1.4.0",
    "spark.jars.excludes": "org.scala-lang:scala-reflect"
   }
}

In [2]:
# Base Configuration
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "champions", 
}
champ = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
champ.count()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574025542235_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')146

In [3]:
matchConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "matches", 
}

match = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**matchConfig).load()

In [4]:
match.count()

2132

In [ ]:
match.head()

In [ ]:
match.createOrReplaceTempView("table1")
match2=spark.sql("SELECT stats.gameType from table1 where stats.gameMode='CLASSIC'")
match2.collect()

### Get Summoner's Rift Matches

In [12]:
srmatch=spark.sql("SELECT * from table1 where stats.gameMode='CLASSIC' and stats.gameDuration>420")
srmatch.count()


1627

## Load Player Data From Player Container

In [85]:
playConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "players", 
}

play = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**playConfig).load()

In [12]:
play.count()

541

In [ ]:
play.head()

In [ ]:
play.createOrReplaceTempView("table2")
play2=spark.sql("SELECT mastery, id from table2")
play2.collect()

In [ ]:
play.printSchema()

In [ ]:
play2.select('mastery')

## Try adding new stats normalized by game duration

In [ ]:
match3=match2.withColumn('damageperminute', participants.stats.totalDamageDealtToChampions/stats.gameDuration)

In [ ]:
srmatch.printSchema()

In [ ]:
srmatch.withColumn('stats.participants.stats.damageperminute', srmatch.stats.particpants.stats.totalDamageDealtToChampions/srmatch.stats.gameDuration)

In [43]:
srmatch.stats.particpants.stats.totalDamageDealtToChampions

Column<b'stats[particpants][stats][totalDamageDealtToChampions]'>

In [54]:
expjoin=srmatch.join(play)

In [60]:
expflat=srmatch.select(srmatch.columns("stats.*"))

'list' object is not callable
Traceback (most recent call last):
TypeError: 'list' object is not callable



In [69]:
flat_cols = [c[0] for c in srmatch.dtypes if c[1][:6] != 'struct']
nested_cols = [c[0] for c in srmatch.dtypes if c[1][:6] == 'struct']
flat_df = srmatch.select(*flat_cols, *[c + ".*" for c in nested_cols])

In [ ]:
flat_df.printSchema()

In [75]:
flat_cols2 = [c[0] for c in flat_df.dtypes if c[1][:20] != 'struct']
nested_cols2 = [c[0] for c in flat_df.dtypes if c[1][:20] == 'struct']
flat_df2 = srmatch.select(*flat_cols, *[c + ".*" for c in nested_cols])

In [ ]:
flat_df2.printSchema()

name 'combine' is not defined
Traceback (most recent call last):
NameError: name 'combine' is not defined



In [6]:
expdf = srmatch.select(pyspark.sql.functions.col('stats.gameDuration').alias('gameDuration'),
                       pyspark.sql.functions.col('stats.participants.stats').alias('participants.stats'))
expdf.show()


name 'srmatch' is not defined
Traceback (most recent call last):
NameError: name 'srmatch' is not defined



In [7]:
import pyspark.sql.functions

In [ ]:
expdf.collect()

In [ ]:
expdf.printSchema()

In [13]:
expdf2 = srmatch.select(pyspark.sql.functions.col('stats.gameDuration').alias('gameDuration'), 
                      pyspark.sql.functions.col('stats.participants.stats.participantId').alias('participantId'), 
                      pyspark.sql.functions.col('stats.participants.stats.totalDamageDealtToChampions').alias('totalDamageDealtToChampions'))

expdf2.printSchema()

root
 |-- gameDuration: integer (nullable = true)
 |-- participantId: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- totalDamageDealtToChampions: array (nullable = true)
 |    |-- element: integer (containsNull = false)

In [ ]:
expdf2.show()

In [18]:
from pyspark.sql.types import IntegerType, ArrayType, DoubleType

def normalise(x, time):
    return [element / (time/60) for element in x]

expdf3=expdf2.withColumn('dpm', pyspark.sql.functions.udf(normalise, ArrayType(DoubleType()))(expdf2['totalDamageDealtToChampions'], 
                                                                                               expdf2['gameDuration']))

In [19]:
expdf3.printSchema()

root
 |-- gameDuration: integer (nullable = true)
 |-- participantId: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- totalDamageDealtToChampions: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- dpm: array (nullable = true)
 |    |-- element: double (containsNull = true)

In [ ]:
expdf3.show()

In [ ]:
srmatch.printSchema()

In [ ]:
expdf4 = srmatch.select(pyspark.sql.functions.col('id'),
                      pyspark.sql.functions.col('stats.gameDuration').alias('gameDuration'), 
                      pyspark.sql.functions.col('stats.participants.stats.participantId').alias('participantId'),
                      pyspark.sql.functions.col('stats.participantIdentities.player.summonerID').alias('summonerID'),
                      pyspark.sql.functions.col('stats.participants.championID').alias('championID'),
                      pyspark.sql.functions.col('stats.participants.teamId').alias('teamId'),
                      pyspark.sql.functions.col('stats.participants.stats.win').alias('win'),  
                      pyspark.sql.functions.col('stats.participants.stats.kills').alias('kills'),
                      pyspark.sql.functions.col('stats.participants.stats.assists').alias('assists'),
                      pyspark.sql.functions.col('stats.participants.stats.deaths').alias('deaths'),
                      pyspark.sql.functions.col('stats.participants.stats.totalDamageDealtToChampions').alias('totalDamageDealtToChampions'),  
                      pyspark.sql.functions.col('stats.participants.stats.visionScore').alias('visionScore'),
                      pyspark.sql.functions.col('stats.participants.stats.totalTimeCrowdControlDealt').alias('totalTimeCrowdControlDealt')
                      )

expdf4.printSchema()

In [36]:
expdf4.show(1)

+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+
|        id|gameDuration|       participantId|          summonerID|          championID|              teamId|                 win|               kills|             assists|              deaths|totalDamageDealtToChampions|         visionScore|totalTimeCrowdControlDealt|
+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+
|3207597356|        1353|[1, 2, 3, 4, 5, 6...|[SAim3CJlYb3gTey9...|[245, 13, 114, 25...|[100, 100, 100, 1...|[false, false, fa...|[3, 2, 0, 4, 1, 1...|[3, 3, 0, 3, 4, 5...|[4, 7, 4, 6, 5, 3.

In [ ]:
expdf5=expdf4.withColumn('dpm', pyspark.sql.functions.udf(normalise, ArrayType(DoubleType()))(expdf4['totalDamageDealtToChampions'], 
                                                                                               expdf4['gameDuration']))
expdf5.printSchema()


In [39]:
expdf5.show(1)

+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+--------------------+
|        id|gameDuration|       participantId|          summonerID|          championID|              teamId|                 win|               kills|             assists|              deaths|totalDamageDealtToChampions|         visionScore|totalTimeCrowdControlDealt|                 dpm|
+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+--------------------+
|3207597356|        1353|[1, 2, 3, 4, 5, 6...|[SAim3CJlYb3gTey9...|[245, 13, 114, 25...|[100, 100, 100, 1...|[false, false, fa.

In [ ]:
expdf5=expdf5.withColumn('vpm', pyspark.sql.functions.udf(normalise, ArrayType(DoubleType()))(expdf4['visionScore'],
                                                                                             expdf4['gameDuration']))
expdf5=expdf5.withColumn('ccpm', pyspark.sql.functions.udf(normalise, ArrayType(DoubleType()))(expdf4['totalTimeCrowdControlDealt'],
                                                                                             expdf4['gameDuration']))
expdf5.printSchema()

In [43]:
expdf5.show(1)

+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+--------------------+--------------------+--------------------+
|        id|gameDuration|       participantId|          summonerID|          championID|              teamId|                 win|               kills|             assists|              deaths|totalDamageDealtToChampions|         visionScore|totalTimeCrowdControlDealt|                 dpm|                 vpm|                ccpm|
+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------------+--------------------+--------------------+--------------------+
|

In [ ]:
champ.createOrReplaceTempView("table3")
champ2=spark.sql("SELECT id, name, stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel FROM table3")
champ2.collect()

In [ ]:
champ2.select("id","name","armorperlevel","hpperlevel","spellblockperlevel","attackdamageperlevel","attackspeed","attackrange").collect()



In [47]:
import numpy as np

In [48]:
champ_array = np.array(champ2.select("id","name","armorperlevel","hpperlevel","spellblockperlevel","attackdamageperlevel","attackspeed","attackrange").collect())

In [49]:
champ_array

array([['266', 'Aatrox', '3.25', ..., '5.0', '0.651', '175'],
       ['103', 'Ahri', '3.5', ..., '3.0', '0.668', '550'],
       ['84', 'Akali', '3.5', ..., '3.3', '0.625', '125'],
       ..., 
       ['26', 'Zilean', '3.8', ..., '3.0', '0.625', '550'],
       ['142', 'Zoe', '3.5', ..., '3.3', '0.625', '550'],
       ['143', 'Zyra', '3.0', ..., '3.2', '0.625', '575']], 
      dtype='<U12')

In [50]:
idl=[1,2,3,4,5]
adf=spark.sql("SELECT stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel WHERE id=x FROM table3") for x in idl

invalid syntax (<stdin>, line 2)
  File "<stdin>", line 2
    adf=spark.sql("SELECT stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel WHERE id=x FROM table3") for x in idl
                                                                                                                                                                                       ^
SyntaxError: invalid syntax



In [51]:
adf.show(1)

name 'adf' is not defined
Traceback (most recent call last):
NameError: name 'adf' is not defined



In [ ]:
def ch_array(idl):
    return spark.sql("SELECT stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel WHERE id=x FROM table3") for x in idl
    return [element / (time/60) for element in x]

In [ ]:
splits=np.vsplit(champ_array, 146)
splits

In [ ]:
for x in splits:
        print(x)

In [83]:
arrays=[]
for x in splits:
    arrays.append(x[0])
arrays

[array(['266', 'Aatrox', '3.25', '90', '1.25', '5.0', '0.651', '175'], 
      dtype='<U12'), array(['103', 'Ahri', '3.5', '92', '0.5', '3.0', '0.668', '550'], 
      dtype='<U12'), array(['84', 'Akali', '3.5', '95', '1.25', '3.3', '0.625', '125'], 
      dtype='<U12'), array(['12', 'Alistar', '3.5', '106', '1.25', '3.62', '0.625', '125'], 
      dtype='<U12'), array(['32', 'Amumu', '3.8', '84', '1.25', '3.8', '0.736', '125'], 
      dtype='<U12'), array(['34', 'Anivia', '4.0', '82', '0.5', '3.2', '0.625', '600'], 
      dtype='<U12'), array(['1', 'Annie', '4.0', '88', '0.5', '2.625', '0.579', '625'], 
      dtype='<U12'), array(['22', 'Ashe', '3.4', '85', '0.5', '2.96', '0.658', '600'], 
      dtype='<U12'), array(['136', 'AurelionSol', '3.6', '92', '0.5', '3.2', '0.625', '550'], 
      dtype='<U12'), array(['268', 'Azir', '3.0', '92', '0.5', '2.8', '0.625', '525'], 
      dtype='<U12'), array(['432', 'Bard', '4.0', '89', '0.5', '3.0', '0.625', '500'], 
      dtype='<U12'), array(['53'

In [84]:
for x in arrays:
    if x[1] == "Ahri":
        print(x)

['103' 'Ahri' '3.5' '92' '0.5' '3.0' '0.668' '550']

In [87]:
play.createOrReplaceTempView("table2")
play2=spark.sql("SELECT mastery, id from table2")
play2.show(1)

+--------------------+--------------------+
|             mastery|                  id|
+--------------------+--------------------+
|[31119, 218, 2712...|EuXHkripW2Qav3H2l...|
+--------------------+--------------------+
only showing top 1 row

In [ ]:

play2=spark.sql("SELECT id, mastery, matches from table2")
play2.show(2)

In [91]:
play2.head()

Row(id='EuXHkripW2Qav3H2l1uGWapRCMgwQh4fHIVFybcTCuTk8XX_', mastery=Row(1=31119, 10=218, 101=2712, 102=None, 103=5543, 104=None, 105=4496, 106=None, 107=None, 11=None, 110=None, 111=None, 112=None, 113=None, 114=591, 115=544, 117=42593, 119=899, 12=None, 120=None, 121=6108, 122=None, 126=1187, 127=None, 13=25608, 131=None, 133=979, 134=436, 136=98, 14=None, 141=None, 142=4003, 143=None, 145=95146, 15=None, 150=None, 154=None, 157=None, 16=45287, 161=1983, 163=None, 164=4769, 17=90, 18=1124, 19=None, 2=772, 20=1650, 201=79, 202=None, 203=None, 21=1632, 22=None, 222=30852, 223=None, 23=None, 235=None, 236=5619, 238=None, 24=20526, 240=None, 245=94233, 246=12755, 25=4923, 254=None, 26=1398, 266=10991, 267=72403, 268=153, 27=None, 28=None, 29=1212, 3=None, 30=None, 31=None, 32=940, 33=None, 34=None, 35=None, 350=4908, 36=None, 37=999, 38=24112, 39=33503, 4=None, 40=27385, 41=None, 412=778, 42=2163, 420=None, 421=976, 427=None, 429=235, 43=11602, 432=968, 44=541, 45=None, 48=None, 497=2871, 

In [92]:
play3 = spark.sql("SELECT id, mastery from table2")

In [ ]:
pyspark.sql.functions.col('mastery').keys()

In [96]:
play3.printSchema()

root
 |-- id: string (nullable = true)
 |-- mastery: struct (nullable = true)
 |    |-- 1: integer (nullable = true)
 |    |-- 10: integer (nullable = true)
 |    |-- 101: integer (nullable = true)
 |    |-- 102: integer (nullable = true)
 |    |-- 103: integer (nullable = true)
 |    |-- 104: integer (nullable = true)
 |    |-- 105: integer (nullable = true)
 |    |-- 106: integer (nullable = true)
 |    |-- 107: integer (nullable = true)
 |    |-- 11: integer (nullable = true)
 |    |-- 110: integer (nullable = true)
 |    |-- 111: integer (nullable = true)
 |    |-- 112: integer (nullable = true)
 |    |-- 113: integer (nullable = true)
 |    |-- 114: integer (nullable = true)
 |    |-- 115: integer (nullable = true)
 |    |-- 117: integer (nullable = true)
 |    |-- 119: integer (nullable = true)
 |    |-- 12: integer (nullable = true)
 |    |-- 120: integer (nullable = true)
 |    |-- 121: integer (nullable = true)
 |    |-- 122: integer (nullable = true)
 |    |-- 126: integer (n

In [99]:
expdic = play3.select(pyspark.sql.functions.col('mastery.1'))

expdic.show(5)


+-----+
|    1|
+-----+
|31119|
|21147|
|39832|
| 1355|
|11241|
+-----+
only showing top 5 rows

In [103]:
expdic2=play3.select(pyspark.sql.functions.col('id').alias('summonerID'),
                     pyspark.sql.functions.col('mastery.*'))
expdic2.show(1)

+--------------------+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+---+---+-----+---+----+----+----+----+----+----+-----+----+---+---+---+----+----+----+----+-----+----+----+----+----+-----+----+----+----+---+----+----+---+----+---+----+----+----+----+-----+----+----+----+----+----+-----+----+-----+-----+----+----+----+-----+-----+---+----+----+----+----+----+----+---+----+----+----+----+----+---+-----+-----+----+-----+----+---+----+----+---+----+---+-----+---+---+----+----+----+---+---+----+----+----+----+----+---+----+----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+---+----+---+----+----+----+---+----+----+----+----+---+---+----+----+----+----+----+----+----+
|          summonerID|    1| 10| 101| 102| 103| 104| 105| 106| 107|  11| 110| 111| 112| 113|114|115|  117|119|  12| 120| 121| 122| 126| 127|   13| 131|133|134|136|  14| 141| 142| 143|  145|  15| 150| 154| 157|   16| 161| 163| 164| 17|  18|  19|  2|  20|201| 20

In [105]:
play3.withColumn('explo',pyspark.sql.functions.explode(play3.mastery)).show(1)

"cannot resolve 'explode(table2.`mastery`)' due to data type mismatch: input to function explode should be array or map type, not struct<1:int,10:int,101:int,102:int,103:int,104:int,105:int,106:int,107:int,11:int,110:int,111:int,112:int,113:int,114:int,115:int,117:int,119:int,12:int,120:int,121:int,122:int,126:int,127:int,13:int,131:int,133:int,134:int,136:int,14:int,141:int,142:int,143:int,145:int,15:int,150:int,154:int,157:int,16:int,161:int,163:int,164:int,17:int,18:int,19:int,2:int,20:int,201:int,202:int,203:int,21:int,22:int,222:int,223:int,23:int,235:int,236:int,238:int,24:int,240:int,245:int,246:int,25:int,254:int,26:int,266:int,267:int,268:int,27:int,28:int,29:int,3:int,30:int,31:int,32:int,33:int,34:int,35:int,350:int,36:int,37:int,38:int,39:int,4:int,40:int,41:int,412:int,42:int,420:int,421:int,427:int,429:int,43:int,432:int,44:int,45:int,48:int,497:int,498:int,5:int,50:int,51:int,516:int,517:int,518:int,53:int,54:int,55:int,555:int,56:int,57:int,58:int,59:int,6:int,60:int,61

In [111]:
from pyspark.sql.functions import array, col, explode, struct, lit
def to_long(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("championId"), col(c).alias("points")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.championId", "kvs.points"])

champ_mastery=to_long(expdic2, ["summonerID"])
champ_mastery.show(5)


+--------------------+----------+------+
|          summonerID|championId|points|
+--------------------+----------+------+
|EuXHkripW2Qav3H2l...|         1| 31119|
|EuXHkripW2Qav3H2l...|        10|   218|
|EuXHkripW2Qav3H2l...|       101|  2712|
|EuXHkripW2Qav3H2l...|       102|  null|
|EuXHkripW2Qav3H2l...|       103|  5543|
+--------------------+----------+------+
only showing top 5 rows

In [112]:
champ2.show(1)

+---+------+-------------+--------------------+-----------+-----------+------------------+----------+
| id|  name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+---+------+-------------+--------------------+-----------+-----------+------------------+----------+
|266|Aatrox|         3.25|                 5.0|      0.651|        175|              1.25|        90|
+---+------+-------------+--------------------+-----------+-----------+------------------+----------+
only showing top 1 row

In [115]:
inner_join = champ_mastery.join(champ2, champ_mastery.championId == champ2.id)
inner_join.show(2)

+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|          summonerID|championId|points| id|   name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|EuXHkripW2Qav3H2l...|        51|  5333| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
|RBQZJQA6Acz3VqBqd...|        51| 30573| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
only showing top 2 rows